In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data generated in *1_notmist.ipynb*. It is located at `./data/notMNIST.pickle`

In [3]:
pickle_file = './data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
* data as a flat matrix,
* labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


Tensorflow workflow:
* Describe the computation that shall be performed including the inputs, variables and operations. These get created as nodes over a computation graph.
* Run the operations on the graph by calling `session.run()`, providing the outputs to fetch from the graph taht get returned. 

In [6]:
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    # Input data.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables. Weights and biases.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions over training, validation and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [8]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at setp %d: %f' % (step, l))
            print('Training accuracy: %.lf%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.lf%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.lf%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at setp 0: 19.737356
Training accuracy: 8%
Validation accuracy: 11%
Loss at setp 100: 2.319112
Training accuracy: 72%
Validation accuracy: 71%
Loss at setp 200: 1.866652
Training accuracy: 75%
Validation accuracy: 73%
Loss at setp 300: 1.614657
Training accuracy: 76%
Validation accuracy: 73%
Loss at setp 400: 1.441978
Training accuracy: 77%
Validation accuracy: 74%
Loss at setp 500: 1.314501
Training accuracy: 78%
Validation accuracy: 74%
Loss at setp 600: 1.216008
Training accuracy: 79%
Validation accuracy: 75%
Loss at setp 700: 1.137200
Training accuracy: 79%
Validation accuracy: 75%
Loss at setp 800: 1.072444
Training accuracy: 80%
Validation accuracy: 75%
Test accuracy: 82%
